# Logistic regression for breast cancer

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [14]:
df=pd.read_csv('Breast cancer.csv')
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
corr=df.corr()
print(corr)
corr=np.array(corr)
corr

                         radius_mean  texture_mean  perimeter_mean  area_mean  \
radius_mean                 1.000000      0.323782        0.997855   0.987357   
texture_mean                0.323782      1.000000        0.329533   0.321086   
perimeter_mean              0.997855      0.329533        1.000000   0.986507   
area_mean                   0.987357      0.321086        0.986507   1.000000   
smoothness_mean             0.170581     -0.023389        0.207278   0.177028   
compactness_mean            0.506124      0.236702        0.556936   0.498502   
concavity_mean              0.676764      0.302418        0.716136   0.685983   
concave points_mean         0.822529      0.293464        0.850977   0.823269   
symmetry_mean               0.147741      0.071401        0.183027   0.151293   
fractal_dimension_mean     -0.311631     -0.076437       -0.261477  -0.283110   
radius_se                   0.679090      0.275869        0.691765   0.732562   
texture_se                 -

array([[ 1.00000000e+00,  3.23781891e-01,  9.97855281e-01,
         9.87357170e-01,  1.70581187e-01,  5.06123578e-01,
         6.76763550e-01,  8.22528522e-01,  1.47741242e-01,
        -3.11630826e-01,  6.79090388e-01, -9.73174431e-02,
         6.74171616e-01,  7.35863663e-01, -2.22600125e-01,
         2.05999980e-01,  1.94203623e-01,  3.76168956e-01,
        -1.04320881e-01, -4.26412691e-02,  9.69538973e-01,
         2.97007644e-01,  9.65136514e-01,  9.41082460e-01,
         1.19616140e-01,  4.13462823e-01,  5.26911462e-01,
         7.44214198e-01,  1.63953335e-01,  7.06588569e-03],
       [ 3.23781891e-01,  1.00000000e+00,  3.29533059e-01,
         3.21085696e-01, -2.33885160e-02,  2.36702222e-01,
         3.02417828e-01,  2.93464051e-01,  7.14009805e-02,
        -7.64371834e-02,  2.75868676e-01,  3.86357623e-01,
         2.81673115e-01,  2.59844987e-01,  6.61377735e-03,
         1.91974611e-01,  1.43293077e-01,  1.63851025e-01,
         9.12716776e-03,  5.44575196e-02,  3.52572947e-

### Removing multi-collinearity

In [39]:
x=list(df.columns)
x=x[1:]
p=df.corr().values.tolist()
ind=[i for i in x]
for i in range(len(p)):
    for j in range(i+1,len(p)):
        if(p[i][j]>0.7 and x[i] in ind):
            ind.remove(x[i])
print(ind)

['symmetry_mean', 'texture_se', 'smoothness_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'texture_worst', 'smoothness_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']


In [24]:
X=[]
for i in range(len(df)):
    temp=[1]
    for j in range(1,len(ind)):
        temp.append(df[ind[j]][i])
    X.append(temp)
X=np.array(X)
y=df['diagnosis']
Y=[]
for i in range(len(y)):
    if y[i]=='M':
        Y.append(1)
    else:
        Y.append(0)
Y=np.array(Y)

In [25]:
X

array([[1.      , 0.9053  , 0.006399, ..., 0.2654  , 0.4601  , 0.1189  ],
       [1.      , 0.7339  , 0.005225, ..., 0.186   , 0.275   , 0.08902 ],
       [1.      , 0.7869  , 0.00615 , ..., 0.243   , 0.3613  , 0.08758 ],
       ...,
       [1.      , 1.075   , 0.005903, ..., 0.1418  , 0.2218  , 0.0782  ],
       [1.      , 1.595   , 0.006522, ..., 0.265   , 0.4087  , 0.124   ],
       [1.      , 1.428   , 0.007189, ..., 0.      , 0.2871  , 0.07039 ]])

In [27]:
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [40]:
Xt=np.transpose(X)
XtXi=np.linalg.inv(Xt.dot(X))
XtY=Xt.dot(Y)
W=XtXi.dot(XtY)
Wt=np.transpose(W)
wtx=[]
for i in range(len(X)):
    for j in range(len(X[0])):
        wtx.append(Wt.dot(X[i]))
wtx

[1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 1.107804802034557,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 0.7450198224801234,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.108804192647204,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 1.0657188634446402,
 0.5591390188568959,
 0.5591390188568959,
 0.5591390188568959,
 0.5591390188568959,
 0.55913901885

In [45]:
wold=wnew=[0]*len(ind)
count=0
while (count==0 or wold==wnew):
    